In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Pre-Processing

## Data Importing

In [2]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
#path = '/content/drive/MyDrive/O4DS - Project Work/goodreads_cleaned.csv'
path = 'DATA/goodreads_cleaned.csv'

In [3]:
df = pd.read_csv(path, sep = ";")
df

,user_id,book_id,rating
0,8842281e1d1347389f2ab93d60773d4d,18245960,5
1,8842281e1d1347389f2ab93d60773d4d,16981,3
2,8842281e1d1347389f2ab93d60773d4d,28684704,3
3,8842281e1d1347389f2ab93d60773d4d,27161156,0
4,8842281e1d1347389f2ab93d60773d4d,25884323,4
...,...,...,...
899995,b9450d1c1f97f891c392b1105959b56e,11832081,3
899996,b9450d1c1f97f891c392b1105959b56e,16095092,3
899997,b9450d1c1f97f891c392b1105959b56e,8430896,4
899998,b9450d1c1f97f891c392b1105959b56e,12275680,4


## Data Exploration

In [4]:
df.user_id.value_counts().describe()

count    12188.000000
mean        73.843124
std        103.860677
min          1.000000
25%         14.000000
50%         37.000000
75%         92.000000
max       1815.000000
Name: user_id, dtype: float64

In [5]:
df.book_id.value_counts().describe()

count    25474.000000
mean        35.330141
std         67.222413
min          1.000000
25%         10.000000
50%         17.000000
75%         34.000000
max       1734.000000
Name: book_id, dtype: float64

## Data Cleaning

In [6]:
df['book_id_count'] = df.groupby('book_id')['book_id'].transform('count')
df['user_id_count'] = df.groupby('user_id')['user_id'].transform('count')
df.book_id_count.quantile(0.9)

457.0

In [7]:
book_quantile = 0.9
user_quantile = 0.25

df = df.loc[(df.book_id_count >= df.book_id.value_counts().quantile(book_quantile)) & (df.user_id_count >= df.user_id.value_counts().quantile(user_quantile)),:]


In [8]:
df.shape

(425794, 5)

## Data Pivoting

In [9]:
df = pd.pivot_table(df, columns="book_id", index="user_id", values="rating")
df.head(100)

book_id,1,2,3,5,6,11,34,295,320,343,350,662,667,830,865,890,902,930,960,968,1103,1232,1420,1617,1618,1622,1845,1852,1885,1934,1953,2052,2156,2165,2187,2493,2526,2612,2623,2657,2744,2839,2956,2998,3008,3431,3473,3636,3682,3758,...,30269126,30312891,30325011,30415154,30555488,30633337,30653853,30687916,30688435,30724132,30731416,30747137,30809689,30821598,30831912,30839185,30969741,31140847,31145133,31145148,31176886,31423196,31450752,31450852,31450908,31451174,31538614,31538635,31538647,31931941,31952703,32075662,32075671,32078787,32571395,32796253,32848471,33140405,33151805,33232571,33280872,33288638,33385229,33643994,34044126,34076952,34273458,35247769,35404657,35504431
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
000a1016fda6008d1edbba720ca00851,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0011e1a9112b3d798702ef5b20bbf35b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0019de4561419b7543238e0979f2f33e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00204424763e8233c5f53f0729f2304f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00214d8b0a020837cccf5f41eb563037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00238d8a4c276c47f5d5e242f54a8f28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
002a023d3de233b4bd3ec4fc3e9c581a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
002e063d40ae0107a59d8f9c1aa7a423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

## Convert to an array to work with the matrix

In [10]:
df.reset_index(drop=True)
df

book_id,1,2,3,5,6,11,34,295,320,343,350,662,667,830,865,890,902,930,960,968,1103,1232,1420,1617,1618,1622,1845,1852,1885,1934,1953,2052,2156,2165,2187,2493,2526,2612,2623,2657,2744,2839,2956,2998,3008,3431,3473,3636,3682,3758,...,30269126,30312891,30325011,30415154,30555488,30633337,30653853,30687916,30688435,30724132,30731416,30747137,30809689,30821598,30831912,30839185,30969741,31140847,31145133,31145148,31176886,31423196,31450752,31450852,31450908,31451174,31538614,31538635,31538647,31931941,31952703,32075662,32075671,32078787,32571395,32796253,32848471,33140405,33151805,33232571,33280872,33288638,33385229,33643994,34044126,34076952,34273458,35247769,35404657,35504431
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
000a1016fda6008d1edbba720ca00851,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0011e1a9112b3d798702ef5b20bbf35b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0019de4561419b7543238e0979f2f33e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00204424763e8233c5f53f0729f2304f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00214d8b0a020837cccf5f41eb563037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff3a250fbc018ad2c2c2d45c86734da,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fff7bfd82b89fa347edfe9a82ac0c61b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [11]:
data_matrix = df.to_numpy(na_value=np.nan)
print(data_matrix)

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan  5. nan ... nan nan nan]]


In [12]:
# Check how to get the index of not empty values
idx = np.argwhere(~np.isnan(data_matrix))
print(idx)

[[   0  419]
 [   0  427]
 [   0  495]
 ...
 [9152 2452]
 [9152 2475]
 [9152 2559]]


In [16]:
np.unique(data_matrix[idx[:,0], idx[:,1]])

array([0., 1., 2., 3., 4., 5.])

# Frank-Wolfe - standard algorithm

- Should we feed $\delta$ to the FW algorithm or should it be defined based on the dimensions of the data?
- Which is the correct objective function?
- Initialize with random matrix of integers from 1 to 5 or with zeros matrix?

In [3]:
from scipy import sparse
from scipy import stats

In [4]:
def FW_objective_function(diff_vec):
    return 0.5*(np.power(diff_vec,2).sum())
    #return 0.5 * np.linalg.norm(diff_vec, 2)**2

def FrankWolfe(X, objective_function, Z_init=None, max_iter=150, patience=1e-3):
    '''
    :param X: sparse matrix with ratings and 'empty values', rows - users, columns - books.
    :param objective_function: objective function that we would like to minimize with FW
    :param Z_init: In case we want to initialize Z with a known matrix, if not given Z_init will be a zeros matrix
    :param max_iter: max number of iterations for the method
    :param patience: once reached this tolerance provide the result
    :return: Z: matrix of predicted ratings - it should be like X but with no 'empty values'
            accuracy: difference between original values (X) and predicted ones (Z)
    '''

    # Get X indexes for not empty values
    idx_ratings = np.argwhere(X != 0)
    #idx_ratings = np.argwhere(~np.isnan(X))
    idx_rows = idx_ratings[:,0]
    idx_cols = idx_ratings[:,1]

    # Initialize Z -- think about a good init
    if Z_init is not None:
        Z = Z_init
    else:
        #Z = np.random.randint(1, 6, size=X.shape)
        #Z = Z.astype(float)
        Z = np.zeros(X.shape)

    # Create vectors with the not empty features of the sparse matrix
    X_rated = X[idx_rows, idx_cols]
    Z_rated = Z[idx_rows, idx_cols]
    diff_vec = Z_rated - X_rated

    # choose an appropriate delta
    delta = 2.5

    diff_err = patience + 1
    err = objective_function(diff_vec)
    it = 0
    while (diff_err > patience) and (it < max_iter):

        # Gradient
        grad = sparse.csr_matrix((diff_vec, (idx_rows, idx_cols)))

        # SVD
        u_max, s_max, v_max = sparse.linalg.svds(grad, k = 1, which='LM')   # Compute k = 1 singular values, starting from the largest (which = 'LM')

        # Update
        update_Z = -delta*np.outer(u_max,v_max)     # Zk_tilde in the theory

        #alpha - as studied in class
        alpha_k = 2/(it+2)
        Z = (1-alpha_k)*Z + alpha_k*update_Z

        # Error
        diff_vec = Z[idx_rows, idx_cols] - X[idx_rows, idx_cols]
        new_err = objective_function(diff_vec)

        # Improvement at this iteration
        diff_err = np.abs(err - new_err)
        err = new_err

        print('Iteration:', it, 'Err:', err, 'Diff err:', diff_err)

        # Count iteration
        it += 1
    return Z, err

We build a smaller matrix for testing the FW alg, then we will apply it to our data

In [5]:
# Create a random sparse matrix for testing
rvs = stats.randint(1,6).rvs
X_test = sparse.random(1500, 2000,              # shape of the sparse matrix
            density = 0.05,             # density of the sparse matrix
            dtype = np.int32,           # data type
            data_rvs=rvs).toarray()     # distribution

#Normalize the values
X_test_norm = X_test/5

In [7]:
pred_ratings, loss = FrankWolfe(X_test_norm, FW_objective_function, max_iter=1000, patience=1e-7)

Iteration: 0 Err: 32867.551113306174 Diff err: 131.54888669383945
Iteration: 1 Err: 32867.55110671334 Diff err: 6.592832505702972e-06
Iteration: 2 Err: 32867.55110506912 Diff err: 1.6442209016531706e-06
Iteration: 3 Err: 32867.55110441187 Diff err: 6.572518032044172e-07
Iteration: 4 Err: 32867.551104083344 Diff err: 3.2852403819561005e-07
Iteration: 5 Err: 32867.55110389563 Diff err: 1.877124304883182e-07
Iteration: 6 Err: 32867.551103778336 Diff err: 1.1729571269825101e-07
Iteration: 7 Err: 32867.55110370014 Diff err: 7.819471647962928e-08


In [9]:
pred_ratings*5

array([[0.00717123, 0.00808342, 0.00726997, ..., 0.00708487, 0.00882518,
        0.00767076],
       [0.00553995, 0.00624464, 0.00561623, ..., 0.00547323, 0.00681766,
        0.00592585],
       [0.00674869, 0.00760714, 0.00684162, ..., 0.00666742, 0.00830519,
        0.00721879],
       ...,
       [0.00674059, 0.00759801, 0.0068334 , ..., 0.00665941, 0.00829522,
        0.00721012],
       [0.00632474, 0.00712925, 0.00641182, ..., 0.00624857, 0.00778346,
        0.0067653 ],
       [0.00588325, 0.00663161, 0.00596426, ..., 0.0058124 , 0.00724015,
        0.00629306]])

In [82]:
idx_ratings = np.argwhere(~np.isnan(X_test))
idx_rows = idx_ratings[:,0]
idx_cols = idx_ratings[:,1]
pred_ratings[idx_rows,idx_cols]*5

array([0.02274294, 0.01757294, 0.01579721, ..., 0.01853367, 0.01482516,
       0.01638343])

#### Our data prediction

In [ ]:
pred_ratings, loss = FrankWolfe(data_matrix, FW_objective_function, max_iter=10000, patience=1e-5)

# Frank-Wolfe In-face

In [10]:
a = np.array([1,2,1,2,1])
b = np.array([1,2,3,4,5])
np.multiply(a,b)

array([1, 4, 3, 8, 5])

In [ ]:
def FW_objective_function(diff_vec):
    return 0.5*(np.power(diff_vec,2).sum())
    #return 0.5 * np.linalg.norm(diff_vec, 2)**2

def FW_inface(X, objective_function, lower_bound=None, Z_init=None, max_iter=150, patience=1e-3):
    '''
    :param X: sparse matrix with ratings and 'empty values', rows - users, columns - books.
    :param objective_function: objective function that we would like to minimize with FW.
    :param Z_init: In case we want to initialize Z with a known matrix, if not given Z_init will be a zeros matrix.
    :param max_iter: max number of iterations for the method.
    :param patience: once reached this tolerance provide the result.
    :return: Z: matrix of predicted ratings - it should be like X but with no 'empty values'
            loss: difference between original values (X) and predicted ones (Z).
    '''

    # Get X indexes for not empty values
    idx_ratings = np.argwhere(X != 0)
    #idx_ratings = np.argwhere(~np.isnan(X))
    idx_rows = idx_ratings[:,0]
    idx_cols = idx_ratings[:,1]

    # choose an appropriate delta
    delta = 6.01

    # Initialize Z_{-1}
    if Z_init is not None:
        Z = Z_init
    else:
        Z = np.zeros(X.shape)

    # Create vectors with the not empty features of the sparse matrix
    X_rated = X[idx_rows, idx_cols]
    Z_rated = Z[idx_rows, idx_cols]
    diff_vec = Z_rated - X_rated

    # Initial gradient and Z0
    grad = sparse.csr_matrix((diff_vec, (idx_rows, idx_cols)))
    u_max, s_max, v_max = sparse.linalg.svds(grad, k = 1, which='LM')
    Z = -delta*np.outer(u_max,v_max)
    Z_rated = Z[idx_rows, idx_cols]

    # Initialize lower bound on the optimal objective function (f*)
    diff_vec = Z_rated - X_rated
    new_low_bound = np.max((objective_function(diff_vec)+np.multiply(diff_vec,Z_rated)),0)

    # Set L and D constants and gamma1, gamma2 constraints
    L = 1
    D = 2*delta
    gamma1 = 0
    gamma2 = 1

    # Compute first iteration thin SVD

    

    # Additional needed parameters
    diff_err = patience + 1
    err = objective_function(diff_vec)
    it = 0
    while (diff_err > patience) and (it < max_iter):

        # Gradient
        grad = sparse.csr_matrix((diff_vec, (idx_rows, idx_cols)))
        low_bound = new_low_bound

        # SVD
        u_max, s_max, v_max = sparse.linalg.svds(grad, k = 1, which='LM')   # Compute k = 1 singular values, starting from the largest (which = 'LM')

        # Update
        update_Z = -delta*np.outer(u_max,v_max)     # Zk_tilde in the theory

        #alpha - as studied in class
        alpha_k = 2/(it+2)
        Z = (1-alpha_k)*Z + alpha_k*update_Z

        # Error
        diff_vec = Z[idx_rows, idx_cols] - X[idx_rows, idx_cols]
        new_err = objective_function(diff_vec)

        # Improvement at this iteration
        diff_err = np.abs(err - new_err)
        err = new_err

        print('Iteration:', it, 'Err:', err, 'Diff err:', diff_err)

        # Count iteration
        it += 1
    return Z, err

## Sub-Chapter